## 使用tensorflow预测泰坦尼克存活

机器学习的几大框架：
* sklearn：传统浅层机器学习，单机版训练，容易学习使用；
* spark-ml：分布式浅层机器学习，分布式训练，难以在线部署；
* xgboost：传统浅层机器学习的巅峰，单机+分布式部署，容易在线服务；
* tensorflow：深度学习最流行的框架，单机+分布式部署，能够分布式在线部署；

### 导入所需的包

In [1]:
import functools

import numpy as np
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf

In [2]:
train_file_path = "./datas/titanic/train.csv"
test_file_path = "./datas/titanic/test.csv"

### 加载数据

In [3]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [4]:
# 标签列
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [5]:
def get_dataset(file_path):
    """
    构建tensorflow的数据集格式
    """
    dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
    return dataset

# 将train和test的csv，分别加载成tensorflow的对象的格式
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [6]:
# 测试一个批次
examples, labels = next(iter(raw_train_data))
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

EXAMPLES: 
 OrderedDict([('sex', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'male', b'male', b'male', b'male', b'male', b'male', b'female',
       b'male', b'male', b'male', b'male', b'male'], dtype=object)>), ('age', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([28., 49., 28., 19., 34.,  1., 36., 39., 34., 28., 16., 31.],
      dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 0, 0, 0, 1, 4, 1, 1, 1, 0, 4, 1], dtype=int32)>), ('parch', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 0, 0, 0, 0, 1, 0, 5, 0, 0, 1, 1], dtype=int32)>), ('fare', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([227.525,   0.   ,  35.5  ,   6.75 ,  26.   ,  39.688,  26.   ,
        31.275,  21.   ,   8.663,  39.688,  37.004], dtype=float32)>), ('class', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'First', b'Third', b'First', b'Third', b'Second', b'Third',
       b'Second', b'Third', b'Second', b'Third', b'Third', b'Second'],

### 数据预处理

机器学习模型的输入，只能是数字

#### 分类数据

In [8]:
# 分类数据的码表
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [9]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    # 提供码表的特征输入
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [10]:
# 分类特征列
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

#### 连续数据

In [11]:
def process_continuous_data(mean, data):
    # 标准化数据的函数
    data = tf.cast(data, tf.float32) * 1/(2*mean)
    return tf.reshape(data, [-1, 1])

In [12]:
# 提前算好的均值
MEANS = {
    'age' : 29.631308,
    'n_siblings_spouses' : 0.545455,
    'parch' : 0.379585,
    'fare' : 34.385399
}

numerical_columns = []

for feature in MEANS.keys():
    num_col = tf.feature_column.numeric_column(
        feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
    numerical_columns.append(num_col)

In [13]:
# 连续特征列的列表
numerical_columns

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x633528830>, 29.631308)),
 NumericColumn(key='n_siblings_spouses', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x633528830>, 0.545455)),
 NumericColumn(key='parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x633528830>, 0.379585)),
 NumericColumn(key='fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x633528830>, 34.385399))]

#### 创建输入层layer

将这两个特征列的集合相加，并且传给 `tf.keras.layers.DenseFeatures` 从而创建一个进行预处理的输入层。

In [14]:
preprocessing_layer = tf.keras.layers.DenseFeatures(
    categorical_columns+numerical_columns)

### 构建模型

从 `preprocessing_layer` 开始构建 `tf.keras.Sequential`。

In [15]:
# 构建一个DNN模型h(g(f(x)))
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [16]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

### 训练、评估和预测

现在可以实例化和训练模型。

In [17]:
train_data = raw_train_data.shuffle(500)
test_data = raw_test_data

In [18]:
model.fit(train_data, epochs=20)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Epoch 1/20
53/53 [==============================] - 2s 47ms/step - loss: 0.6546 - accuracy: 0.6507
Epoch 2/20
53/53 [==============================] - 0s 5ms/step - loss: 0.5308 - accuracy: 0.8102
Epoch 3/20
53/53 [==============================] - 0s 5ms/step - loss: 0.4572 - accuracy: 0.8086
Epoch 4/20
53/53 [==============================] - 0s 6ms/step - loss: 0.4304 - accuracy: 0.8118
Epoch 5/20
53/53 [==============================] - 0s 4ms/step - loss: 0.4190 - accuracy: 0.8198
Epoch 6/20
53/53 [==============================] - 0s 5ms/step - loss: 0.4200 - accuracy: 0.8214
Epoch 7/20
53/53 [==============================] - 0s 5ms/step - loss: 0.4076 - accuracy: 0.8214
Epoch 8/20
53/53 [==============================] - 0s 4ms/step -

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features (DenseFeature multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  1600      
_________________________________________________________________
dense_1 (Dense)              multiple                  1040      
_________________________________________________________________
dense_2 (Dense)              multiple                  17        
Total params: 2,657
Trainable params: 2,657
Non-trainable params: 0
_________________________________________________________________


当模型训练完成的时候，你可以在测试集 `test_data` 上检查准确性。

In [20]:
test_loss, test_accuracy = model.evaluate(test_data)

print()
print(f'Test Loss {test_loss}, Test Accuracy {test_accuracy}')

     22/Unknown - 1s 25ms/step - loss: 0.4546 - accuracy: 0.7765
Test Loss 0.4546199142932892, Test Accuracy 0.7765151262283325


使用 `tf.keras.Model.predict` 推断一个批次或多个批次的标签。

In [21]:
predictions = model.predict(test_data)
predictions[:10]

array([[0.118],
       [0.193],
       [0.119],
       [0.853],
       [0.088],
       [0.526],
       [0.184],
       [0.115],
       [0.13 ],
       [0.128]], dtype=float32)

In [22]:
list(test_data)[0][1]

<tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0], dtype=int32)>

In [23]:
# 显示部分结果
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    is_survived = "SURVIVED" if bool(survived) else "DIED"
    print(f"预测活着的概率: {prediction[0]} | 实际值: {is_survived}")

预测活着的概率: 0.1184995099902153 | 实际值: DIED
预测活着的概率: 0.19340497255325317 | 实际值: SURVIVED
预测活着的概率: 0.11861217767000198 | 实际值: SURVIVED
预测活着的概率: 0.8527184724807739 | 实际值: DIED
预测活着的概率: 0.08784296363592148 | 实际值: DIED
预测活着的概率: 0.5261467099189758 | 实际值: DIED
预测活着的概率: 0.18388023972511292 | 实际值: DIED
预测活着的概率: 0.11482548713684082 | 实际值: DIED
预测活着的概率: 0.12965388596057892 | 实际值: DIED
预测活着的概率: 0.12832288444042206 | 实际值: DIED
